In [1]:
import os
import lzma
import pickle
from tqdm import tqdm
import datajoint as dj

# Define database prefix

In [2]:
dj.config['database.prefix']='kabilar_'
from pipeline import meso, stack, treadmill, pupil, odor, mice, experiment, shared

Loading local settings from pipeline_config.json
Connecting user1@host.docker.internal:3306


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

DLC loaded in light mode; you cannot use the labeling GUI!
DLC loaded in light mode; you cannot use the relabeling GUI!


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


DataJoint Module Installed =  True


# Define animal_id data to be ingested

In [3]:
animal_id = 1571

In [ ]:
animal_id  = 14864

# Function to insert metadata

In [4]:
directory = f'{os.environ.get("INGESTION_STORAGE")}/animal_id_{animal_id}/'

def insert_data(filename, table, skip_duplicates, ignore_extra_fields, allow_direct_insert):
    with lzma.open(directory + filename, "rb") as f:
        loaded_data = pickle.load(f)
        f.close()

#     print(loaded_data)

    for i in tqdm(range(len(loaded_data))):
        table.insert1(loaded_data[i],
                      skip_duplicates=skip_duplicates,
                      ignore_extra_fields=ignore_extra_fields,
                      allow_direct_insert=allow_direct_insert)

# Mice pipeline

In [5]:
insert_data(filename='mice.Mice.xz', 
            table=mice.Mice, 
            skip_duplicates=False, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

100%|██████████| 1/1 [00:00<00:00, 101.68it/s]


# Shared pipeline

In [6]:
insert_data(filename='shared.PipelineVersion.xz', 
            table=shared.PipelineVersion, 
            skip_duplicates=True, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

100%|██████████| 3/3 [00:00<00:00, 18.94it/s]


# Experiment pipeline

In [7]:
insert_data('experiment.Rig.xz', experiment.Rig, True, False, False)
insert_data('experiment.Lens.xz', experiment.Lens, True, False, False)
insert_data('experiment.Aim.xz', experiment.Aim, True, False, False)
insert_data('experiment.Software.xz', experiment.Software, True, False, False)
insert_data('experiment.Session.xz', experiment.Session, False, True, False)
insert_data('experiment.Scan.xz', experiment.Scan, False, False, False)
insert_data('experiment.Scan.EyeVideo.xz', experiment.Scan.EyeVideo, False, False, False)
insert_data('experiment.Scan.BehaviorFile.xz', experiment.Scan.BehaviorFile, False, False, False)
insert_data('experiment.Scan.Laser.xz', experiment.Scan.Laser, False, False, False)
insert_data('experiment.TreadmillSpecs.xz', experiment.TreadmillSpecs, False, False, False)

100%|██████████| 7/7 [00:00<00:00, 95.62it/s]


In [9]:
experiment.ExperimentalIdentifier.insert(experiment.Scan.fetch('KEY'),skip_duplicates=True)

# Meso pipeline

In [10]:
insert_data(filename='meso.Version.xz', 
            table=meso.Version, 
            skip_duplicates=False, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

100%|██████████| 2/2 [00:00<00:00, 106.85it/s]


In [11]:
meso.ScanInfo.populate()

Reading header...


In [12]:
dj.config["enable_python_native_blobs"] = True
meso.Quality.populate()

Computing quality metrics for field 1
Using 3 processes
Tue Feb 23 06:23:35 2021 Processing frames: slice(0, 1997, None)
Tue Feb 23 06:23:51 2021 Processing frames: slice(1997, 3994, None)
Tue Feb 23 06:24:07 2021 Processing frames: slice(3994, 5991, None)
Tue Feb 23 06:24:23 2021 Processing frames: slice(5991, 7988, None)
Tue Feb 23 06:24:41 2021 Processing frames: slice(7988, 9985, None)
Tue Feb 23 06:25:00 2021 Processing frames: slice(9985, 11982, None)
Tue Feb 23 06:25:18 2021 Processing frames: slice(11982, 13979, None)
Tue Feb 23 06:25:40 2021 Processing frames: slice(13979, 15976, None)
Tue Feb 23 06:25:58 2021 Processing frames: slice(15976, 17973, None)
Tue Feb 23 06:26:16 2021 Processing frames: slice(17973, 19970, None)
Tue Feb 23 06:26:32 2021 Processing frames: slice(19970, 21967, None)
Tue Feb 23 06:26:52 2021 Processing frames: slice(21967, 23964, None)
Tue Feb 23 06:27:12 2021 Processing frames: slice(23964, 25961, None)
Tue Feb 23 06:27:33 2021 Processing frames: slic

In [13]:
insert_data('meso.CorrectionChannel.xz', meso.CorrectionChannel, False, False, False)

  0%|          | 0/3 [00:00<?, ?it/s]


DuplicateError: ("Duplicate entry '1571-1-1-1' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [14]:
meso.RasterCorrection.populate()
meso.MotionCorrection.populate()
meso.SummaryImages.populate()

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


Using 3 processes
Tue Feb 23 07:16:22 2021 Processing frames: slice(0, 3121, None)
Tue Feb 23 07:17:08 2021 Processing frames: slice(3121, 6242, None)
Tue Feb 23 07:18:10 2021 Processing frames: slice(6242, 9363, None)
Tue Feb 23 07:19:14 2021 Processing frames: slice(9363, 12484, None)
Tue Feb 23 07:20:32 2021 Processing frames: slice(12484, 15605, None)
Tue Feb 23 07:21:32 2021 Processing frames: slice(15605, 18726, None)
Tue Feb 23 07:22:36 2021 Processing frames: slice(18726, 21847, None)
Tue Feb 23 07:23:51 2021 Processing frames: slice(21847, 24968, None)
Tue Feb 23 07:25:15 2021 Processing frames: slice(24968, 28089, None)
Tue Feb 23 07:26:10 2021 Processing frames: slice(28089, 31210, None)
Tue Feb 23 07:27:21 2021 Processing frames: slice(31210, 34331, None)
Tue Feb 23 07:28:30 2021 Processing frames: slice(34331, 37452, None)
Tue Feb 23 07:29:22 2021 Processing frames: slice(37452, 40000, None)
Using 3 processes
Tue Feb 23 07:31:33 2021 Processing frames: slice(0, 2913, None)

In [15]:
insert_data('meso.SegmentationTask.xz', meso.SegmentationTask, False, False, False)
insert_data('meso.Segmentation.xz', meso.Segmentation, False, False, True)
insert_data('meso.Segmentation.Manual.xz', meso.Segmentation.Manual, False, False, True)
insert_data('meso.Segmentation.Mask.xz', meso.Segmentation.Mask, False, False, True)

100%|██████████| 183/183 [00:03<00:00, 57.11it/s]


In [5]:
meso.Fluorescence.populate()
meso.MaskClassification.populate()
meso.ScanSet.populate()
meso.Activity.populate()
meso.ScanDone.populate()

Bokeh could not be loaded. Either it is not installed or you are not running within a notebook
numba not found


Using TensorFlow backend.


Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 1, 'channel': 1, 'segmentation_method': 1, 'spike_method': 2}
Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 2, 'channel': 1, 'segmentation_method': 1, 'spike_method': 2}
Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 3, 'channel': 1, 'segmentation_method': 1, 'spike_method': 2}
Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 1, 'channel': 1, 'segmentation_method': 1, 'spike_method': 3}


/usr/local/lib/python3.6/dist-packages/c2s/c2s.py:509: UserWarning: The default model expects calcium traces sampled at 100 Hz.
  warn('The default model expects calcium traces sampled at 100 Hz.')
/usr/local/lib/python3.6/dist-packages/c2s/c2s.py:509: UserWarning: The default model expects calcium traces sampled at 100 Hz.
  warn('The default model expects calcium traces sampled at 100 Hz.')
/usr/local/lib/python3.6/dist-packages/c2s/c2s.py:509: UserWarning: The default model expects calcium traces sampled at 100 Hz.
  warn('The default model expects calcium traces sampled at 100 Hz.')
/usr/local/lib/python3.6/dist-packages/c2s/c2s.py:509: UserWarning: The default model expects calcium traces sampled at 100 Hz.
  warn('The default model expects calcium traces sampled at 100 Hz.')
/usr/local/lib/python3.6/dist-packages/c2s/c2s.py:509: UserWarning: The default model expects calcium traces sampled at 100 Hz.
  warn('The default model expects calcium traces sampled at 100 Hz.')
/usr/local

Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 2, 'channel': 1, 'segmentation_method': 1, 'spike_method': 3}


...

Creating activity traces for {'animal_id': 1571, 'session': 1, 'scan_idx': 1, 'pipe_version': 1, 'field': 1, 'channel': 1, 'segmentation_method': 1, 'spike_method': 5}


/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/deconvolution.py:1001: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/deconvolution.py:1001: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/deconvolution.py:1001: FutureWarning: `rcond` parameter will change to the default of machine pre

PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal
PROBLEM STATUS:optimal
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
TRYING AGAIN ECOS
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:optimal_inaccurate
PROBLEM STATUS:opt

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


LostConnectionError: Connection was lost during a transaction.

# WIP - Automated glomeruli segmentation

In [7]:
entry_segmentation_task=[]

for field in [1,2,3]:
    entry_segmentation_task.append({'animal_id': animal_id,
      'session': 1,
      'scan_idx': 1,
      'field': field,
      'channel': 1,
      'segmentation_method': 2,
      'compartment': 'unknown'})
meso.SegmentationTask.insert(entry_segmentation_task)  

# (meso.SegmentationTask & 'segmentation_method=2').delete()
# (meso.Segmentation & 'segmentation_method=2').delete()

# entry_segmentation={'animal_id': 1571,
#   'session': 1,
#   'scan_idx': 1,
#   'pipe_version': 1,
#   'field': 1,
#   'channel': 1,
#   'segmentation_method': 2,
#   'segmentation_time': '2020-03-23 16:28:40'}
# meso.Segmentation.insert1(entry_segmentation, allow_direct_insert=True)

# entry_segmentation_cnmf={'animal_id': 1571,
#   'session': 1,
#   'scan_idx': 1,
#   'pipe_version': 1,
#   'field': 1,
#   'channel': 1,
#   'segmentation_method': 2,
#   'params': '{"merge_threshold": 0.7, "soma_diameter": [14.0, 14.0], "num_pixels_per_process": 10000, "init_on_patches": false, "num_background_components": 1, "num_components": 720, "num_processes": 12, "init_method": "greedy_roi"}'}
# meso.Segmentation.CNMF.insert1(entry_segmentation_cnmf, allow_direct_insert=True)

# meso.Segmentation.populate()

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


# Odor pipeline

In [6]:
insert_data('odor.Odorant.xz', odor.Odorant, False, False, False)
insert_data('odor.OdorSolution.xz', odor.OdorSolution, False, False, False)
insert_data('odor.OdorSession.xz', odor.OdorSession, False, False, False)
insert_data('odor.OdorConfig.xz', odor.OdorConfig, False, False, False)
insert_data('odor.OdorRecording.xz', odor.OdorRecording, False, False, False)
insert_data('odor.MesoMatch.xz', odor.MesoMatch, False, False, False)

100%|██████████| 1/1 [00:00<00:00, 71.80it/s]


In [7]:
odor.OdorTrials.populate()
odor.OdorSync.populate()
odor.Respiration.populate()

Populating trials for {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0}
1579 odor trials found and inserted for {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0}.

Populating Sync for {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0}
ScanImage sync added for animal 1571, olfactory session 10, recording 0

Populating Respiration for {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0}
Largest NaN gap found: 0.06189109999999687 seconds
Respiration data for {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0} successfully inserted.



In [8]:
odor.OdorAnalysis.populate()

paths init  /data/odor_meso/paths.init
My location is work_1
Datajoint odor connection passes test
kwargs {'paths_init': '/data/odor_meso/paths.init', 'expt_id': 0}
Import Instructions None 0
DataMan ready to build from default DataJoint.
Meso Connected
Datajoint pipelines connected
import using restrictions  0 {'animal_id': 1571, 'odor_session': 10, 'recording_idx': 0, 'session': 1, 'scan_idx': 1, 'pipe_version': 1}
Failure importing all fluorescence and timing data
Halting import early
Import info problems.  Build of DataMan stopped


OSError: Unable to open file (unable to open file: name = '/data/external/meso_storage/scans_stitched.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Treadmill pipeline

In [ ]:
treadmill.Treadmill.populate()

# Stack pipeline - Work in progress

In [ ]:
experiment.Stack.insert1([121,8,1,'meso','unset','stack','scanimage','2017b',0,0,0,'','2020-12-09 11:46:48'])#unsure of depths
experiment.Stack.Filename.insert1([121,8,1,1,'121_8_00001',0])#{animal_id, session, stack_idx, filename_idx, filename, surf_depth}) #unsure of depths

In [ ]:
stack.StackInfo.populate()
stack.Quality.populate()
stack.CorrectionChannel.insert1([121,8,1,1])#correct?
stack.RasterCorrection.populate()
stack.MotionCorrection.populate()
stack.Stitching.populate()
stack.CorrectedStack.populate()
stack.PreprocessedStack.populate()#dj.conn.ping required
# stack.Surface.populate() # did not populate
stack.SegmentationTask.insert1([121,8,1,1,1,2,'soma'])#correct?

In [ ]:
stack.Segmentation.populate()
stack.RegistrationTask.insert1([121,8,1,1,,5])#correct?
stack.Registration.populate()
stack.FieldSegmentation.populate()
stack.RegistrationOverTime.populate()
stack.Drift.populate()
stack.StackSet.populate()
stack.Area.populate()

# Pupil pipeline - Work in progress

In [ ]:
pupil.Eye.populate()

In [ ]:
class TrackingTask(dj.Manual):
pupil.TrackedVideo.populate()
class ManuallyTrackedContours(dj.Manual, AutoPopulate):
pupil.FittedContour.populate()
class ConfigDeeplabcut(dj.Manual):
pupil.Tracking.populate()
pupil.FittedPupil.populate()

# Drop schemas

In [ ]:
mice.schema.drop()
shared.schema.drop()
experiment.schema.drop()
meso.schema.drop()
odor.schema.drop()
treadmill.schema.drop()
stack.schema.drop()
pupil.schema.drop()